In [1]:
# Deep Learning (RNN) Demo for Load Forecasting

### Step 1: Import all the packages needed

In [2]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mp
import argparse
import os, sys
import csv
import math

#### Step 2.1: setting all global parameters -- sec 1 data parameters

In [3]:
data_path = './input.csv'
total_days = 350
train_days = 280
test_days = 70
data_length = 0

#### Step 2.2: setting all global parameters -- sec 2 network configuration

In [4]:
learning_rate = 0.001
num_epoches = 50
steps = 7*48 # input size
batch_size = 70*48 # days of a batch
feature_size = 1 # same time of a week
cell_size = 20 # input size 
num_layers = 3

### Step 3:loading data

In [5]:
dataframe = pd.read_csv(data_path)
dat = np.array(dataframe)
date_list = dat[:,1]
dat = dat[:,2:]# drop the first two cols --- index and date
nrows,ncols = dat.shape
#print nrows,ncols
data = dat.reshape((1,nrows*ncols))
data_length = data.shape[1]

# construct training data
train_len = train_days*48
train_data = data[0,0:train_len]
train_data = train_data.reshape([1,train_len])

# construct testing data
## test size = input_size + test days size. since, the output should be 
## from first test sample to last. prefix is input-size data
test_len = test_days*48
test_data = np.zeros(test_len+steps)
test_data[steps:] = data[0,train_len:train_len+test_len]
test_data[0:steps] = data[0,train_len-steps:train_len]
test_data = test_data.reshape([1,test_len+steps])

### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [6]:
def train_data_gen(steps = 48, n_batch = 48):
    X = np.zeros((n_batch,steps,1))
    Y = np.zeros((n_batch,))
    #for each n, compute X and correct y values
    for n in range(n_batch):
        # randomly pick a sample's y, between acceptable range
        index = np.random.randint(steps,train_len)
        # update y
        Y[n] = train_data[0,index]
        # update X from index-steps to index-1
        X[n,:,:] = train_data[0,index-steps:index].T.reshape(steps,1)
    return (X,Y)

In [7]:
def test_data_gen(steps = 7*48, n_batch = 70*48):
    X = np.zeros((n_batch,steps,1))
    Y = np.zeros((n_batch,))
    #for each n, compute X and correct y values
    for n in range(n_batch):
        # randomly pick a sample's y, between acceptable range
        index = steps
        # update y
        Y[n] = test_data[0,steps+n]
        # update X from index-steps to index-1
        X[n,:,:] = test_data[0,n:n+steps].T.reshape(steps,1)
    return (X,Y)

### Step 5: construct RNN model

In [8]:
#define rnn cell unit
lstm = rnn_cell.BasicLSTMCell(cell_size)
stacked_lstm = rnn_cell.MultiRNNCell([lstm]*num_layers)

# create placeholder for x and y
inputs = [tf.placeholder(tf.float32,shape=[batch_size,feature_size]) for _ in range(steps)]
result = tf.placeholder(tf.float32,shape = [batch_size])

# connect rnn computing graph
outputs, states = rnn.rnn(lstm,inputs,dtype = tf.float32)

# get the last output from list of outputs, then its the output of all steps inputs. SInce the outputs is a list of shape 

# (batch_size,feature_size), 
outputfs = outputs[-1]

#define a linear transform into single outputs
WW = tf.Variable(tf.random_normal([cell_size,1],stddev = 0.01))
bb = tf.Variable(tf.random_normal([1],stddev = 0.01))

#single value output
output = tf.matmul(outputfs,WW) + bb

#cost function
cost = tf.reduce_mean(tf.pow(output-result,2)) # cost function of this batch of data

#compute parameter updates
#train_op = tf.train.GradientDescentOptimizer(0.008).minimize(cost)
train_op = tf.train.RMSPropOptimizer(0.005, 0.3).minimize(cost)



### Step 6: generate validation data

In [9]:
tmpx,y_val = test_data_gen(steps,batch_size)
x_val = []
for i in range(steps):
    x_val.append(tmpx[:,i,:])

### Step 7: run rnn network

In [10]:
### Execute
outp = []
with tf.Session() as sess:
    # Create a summary to monitor cost function
    tf.scalar_summary("loss", cost)
    # Merge all summaries to a single operator
    merged_summary_op = tf.merge_all_summaries()

    # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)
    
    #initialize all variables in the model
    tf.initialize_all_variables().run()
    for k in range(num_epoches):
        #Generate Data for each epoch
        #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
        #this is required to feed data into rnn.rnn
        tempX,Y = train_data_gen(steps,batch_size)
        X = []
        for i in range(steps):
            X.append(tempX[:,i,:])

        #Create the dictionary of inputs to feed into sess.run
        temp_dict = {inputs[i]:X[i] for i in range(steps)}
        temp_dict.update({result: Y})
        sess.run(train_op,feed_dict=temp_dict)   #perform an update on the parameters
        
        val_dict = {inputs[i]:x_val[i] for i in range(steps)}  #create validation dictionary
        val_dict.update({result: y_val})
        c_val = sess.run(cost, feed_dict = val_dict )            #compute the cost on the validation set
        output_tmp = sess.run(output,feed_dict = val_dict)
        outp = output_tmp
        # Write logs at every iteration
        summary_str = sess.run(merged_summary_op, feed_dict=val_dict)
        summary_writer.add_summary(summary_str, k)
        print "Validation cost: {}, on Epoch {}".format(c_val,k)
    print "haha{}".format(outp)

Validation cost: 0.168670088053, on Epoch 0
Validation cost: 0.16671000421, on Epoch 1
Validation cost: 0.164190962911, on Epoch 2
Validation cost: 0.161644503474, on Epoch 3
Validation cost: 0.158691421151, on Epoch 4
Validation cost: 0.155087605119, on Epoch 5
Validation cost: 0.150876179338, on Epoch 6
Validation cost: 0.145681932569, on Epoch 7
Validation cost: 0.139449611306, on Epoch 8
Validation cost: 0.13312163949, on Epoch 9
Validation cost: 0.128440305591, on Epoch 10
Validation cost: 0.127058327198, on Epoch 11
Validation cost: 0.126752585173, on Epoch 12
Validation cost: 0.126983523369, on Epoch 13
Validation cost: 0.128652691841, on Epoch 14
Validation cost: 0.127254664898, on Epoch 15
Validation cost: 0.127430990338, on Epoch 16
Validation cost: 0.127287685871, on Epoch 17
Validation cost: 0.127505093813, on Epoch 18
Validation cost: 0.127254396677, on Epoch 19
Validation cost: 0.127468377352, on Epoch 20
Validation cost: 0.12720015645, on Epoch 21
Validation cost: 0.1276

### Step 8: Evaluation

In [11]:
a = [1,2,3]
b = [1,3,4]
a = np.array(a)
b = np.array(b)

In [12]:
np.corrcoef(a,b)[0,1]

0.98198050606196585

In [13]:
out_p = np.array(outp)

In [14]:
out_p = out_p.reshape([3360,])

In [15]:
out_p.shape

(3360,)

In [16]:
out_p

array([ 0.1941347 ,  0.19437268,  0.19445409, ...,  0.19401573,
        0.19438878,  0.19458491], dtype=float32)

In [17]:
y_val.dtype = float
y_val

array([ 0.039,  0.086,  0.029, ...,  0.512,  0.53 ,  0.611])

In [18]:
R = np.corrcoef(out_p,y_val)
RR = R**2

In [19]:
R

array([[ 1.        , -0.25146815],
       [-0.25146815,  1.        ]])

In [20]:
RR

array([[ 1.        ,  0.06323623],
       [ 0.06323623,  1.        ]])

In [21]:
import matplotlib.pyplot as pl
xxx = np.arange(0,test_len)
#pl.plot(xxx,outp,label = 'predict',color = 'red')
pl.plot(xxx,y_val,label = 'reality',color = 'blue')
pl.grid()
pl.legend()
pl.show()

In [22]:
sq = pow(out_p-y_val,2)

In [23]:
np.mean(sq)

0.12768098082839643